In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile, move, rmtree

### Data Preparation

In [ ]:
!unzip /kaggle/input/dogs-vs-cats/train.zip

In [ ]:
len(os.listdir('/kaggle/working/train'))

In [ ]:
os.listdir('/kaggle/working/train')[:5]

In [ ]:
base_directory = '/kaggle/working/pet-images'
cat_directory = os.path.join(base_directory, 'cats')
dog_directory = os.path.join(base_directory, 'dogs')

try:
    os.mkdir(base_directory)
    os.mkdir(cat_directory)
    os.mkdir(dog_directory)
except:
    pass

In [ ]:
os.listdir('/kaggle/working/pet-images')

In [ ]:
source = '/kaggle/working/train'

for image in os.listdir(source):
    if image.split('.')[0] == 'cat':
        move(os.path.join(source, image), os.path.join(cat_directory, image))
    else:
        move(os.path.join(source, image), os.path.join(dog_directory, image))

In [ ]:
len(os.listdir('/kaggle/working/train'))

In [ ]:
print(f'# of cat images: {len(os.listdir(cat_directory))}')
print(f'# of dog images: {len(os.listdir(dog_directory))}')

In [ ]:
try:
    os.mkdir("/kaggle/working/cats-vs-dogs/")
    os.mkdir("/kaggle/working/cats-vs-dogs/training/")
    os.mkdir("/kaggle/working/cats-vs-dogs/training/cats/")
    os.mkdir("/kaggle/working/cats-vs-dogs/training/dogs/")
    os.mkdir("/kaggle/working/cats-vs-dogs/validation/")
    os.mkdir("/kaggle/working/cats-vs-dogs/validation/cats/")
    os.mkdir("/kaggle/working/cats-vs-dogs/validation/dogs")
except OSError:
    pass

In [ ]:
print(os.listdir('/kaggle/working/cats-vs-dogs'))
print(os.listdir('/kaggle/working/cats-vs-dogs/training'))
print(os.listdir('/kaggle/working/cats-vs-dogs/validation'))

In [ ]:
def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
    source_files = os.listdir(SOURCE)
    randomized_source_files = random.sample(source_files, len(source_files))

    train_split = randomized_source_files[ :int(SPLIT_SIZE * len(source_files))]
    valid_split = randomized_source_files[int(SPLIT_SIZE * len(source_files)): ]

    for file in train_split:
        if os.path.getsize(os.path.join(SOURCE, file)) != 0:
            copyfile(os.path.join(SOURCE, file), os.path.join(TRAINING, file))
        else:
            print(f"{file} has size 0, so ignoring")
    
    for file in valid_split:
        if os.path.getsize(os.path.join(SOURCE, file)) != 0:
            copyfile(os.path.join(SOURCE, file), os.path.join(VALIDATION, file))
        else:
            print(f"{file} has size 0, so ignoring")

In [ ]:
CAT_SOURCE_DIR = cat_directory
TRAINING_CATS_DIR = "/kaggle/working/cats-vs-dogs/training/cats/"
VALIDATION_CATS_DIR = "/kaggle/working/cats-vs-dogs/validation/cats/"

DOG_SOURCE_DIR = dog_directory
TRAINING_DOGS_DIR = "/kaggle/working/cats-vs-dogs/training/dogs/"
VALIDATION_DOGS_DIR = "/kaggle/working/cats-vs-dogs/validation/dogs/"

split_size = .9

split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)

In [ ]:
print("# of cats in train directory", len(os.listdir(TRAINING_CATS_DIR)))
print("# of cats in validation directory", len(os.listdir(VALIDATION_CATS_DIR)))
print("# of dogs in train directory", len(os.listdir(TRAINING_DOGS_DIR)))
print("# of dogs in validation directory", len(os.listdir(VALIDATION_DOGS_DIR)))

### Preparing the Data Generators

In [ ]:
TRAINING_DIR = "/kaggle/working/cats-vs-dogs/training/"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (150,150),
    batch_size = 128,
    class_mode = 'binary'
)

VALIDATION_DIR = "/kaggle/working/cats-vs-dogs/validation/"

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (150,150),
    batch_size = 128,
    class_mode = 'binary'
)

### Building Model from Scratch

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        if (logs.get('val_accuracy') > 0.96):
            print('validation Accuracy reached 96% , so Stop now')
            self.model.stop_training = True

In [ ]:
history = model.fit(
            train_generator,
            epochs=5,
            verbose=1,
            validation_data=validation_generator,
            callbacks=[myCallback()])

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

### Using Transfer Learning

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
pre_trained_model = InceptionV3(input_shape=(150,150,3),
                               include_top=False,
                               weights='imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model2 = tf.keras.Model(pre_trained_model.input,x)
model2.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
history2 = model2.fit(
            train_generator,
            epochs=5,
            verbose=1,
            validation_data=validation_generator,
            callbacks=[myCallback()])

In [ ]:
pre_trained_model.trainable = True

model2.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

model2.summary()

In [ ]:
history2 = model2.fit(
            train_generator,
            epochs=5,
            verbose=1,
            validation_data=validation_generator,
            callbacks=[myCallback()])

### Testing

In [ ]:
import PIL
from PIL import Image

In [ ]:
cat1 = Image.open('/kaggle/input/pettest/leia.jpg')
cat1 = cat1.resize((150, 150))
cat2 = Image.open('/kaggle/input/pettest/luke.jpg')
cat2 = cat2.resize((150, 150))
dog1 = Image.open('/kaggle/input/pettest/marley.jpg')
dog1 = dog1.resize((150, 150))

In [ ]:
source = '/kaggle/input/pettest'
destination = '/kaggle/working/test/'
cat_destination = '/kaggle/working/test/cat'
dog_destination = '/kaggle/working/test/dog'

try:
    os.mkdir(destination)
    os.mkdir(cat_destination)
    os.mkdir(dog_destination)
except:
    pass

for image in os.listdir(source):
    if image.split('.')[0] == 'marley':
        copyfile(os.path.join(source, image), os.path.join(dog_destination, image))
    else:
        copyfile(os.path.join(source, image), os.path.join(cat_destination, image))

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

destination = '/kaggle/working/test'

test_generator = test_datagen.flow_from_directory(destination,
                                                  target_size=(150,150),
                                                  batch_size=3,
                                                  class_mode='binary')
x, y = next(test_generator)

In [ ]:
predictions = model2.predict(x)
predicted_val = [int(round(p[0])) for p in predictions]

In [ ]:
for i in range(3):
    image = x[i]
    plt.imshow(image)
    print("Cat") if predicted_val[i] == 0 else print("Dog")
    plt.show()